In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Load datasets
train_df = pd.read_csv("/content/X_train.csv")
test_df  = pd.read_csv("/content/X_test.csv")

# Separate target
y_train = train_df['SalePrice']
X_train = train_df.drop(['Id', 'SalePrice'], axis=1)
X_test  = test_df.drop(['Id'], axis=1)

# Align columns (keep only features)
all_features = list(set(X_train.columns) | set(X_test.columns))
X_train = X_train.reindex(columns=all_features, fill_value=0)
X_test  = X_test.reindex(columns=all_features, fill_value=0)

# Fill missing values in train (mean imputation)
X_train = X_train.fillna(X_train.mean())
X_test  = X_test.fillna(X_train.mean())  # Use train's stats

# Feature selection with Lasso
feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=42))
feature_sel_model.fit(X_train, y_train)
selected_feat = X_train.columns[feature_sel_model.get_support()]

X_train = X_train[selected_feat]
X_test  = X_test[selected_feat]

# Split train into training/validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Train linear regression
model = LinearRegression()
model.fit(X_tr, y_tr)

# Predict on validation set
y_pred = model.predict(X_val)

# Metrics
r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"R-squared (R2 Score): {r2:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


R-squared (R2 Score): 0.92
Mean Squared Error (MSE): 0.02
Root Mean Squared Error (RMSE): 0.12
